In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import pandas as pd
from econ_evals.utils.helper_functions import get_base_dir_path
from ast import literal_eval
from econ_evals.experiments.pricing.pricing_market_logic_multiproduct import (
    get_monopoly_prices,
    get_nash_prices,
    get_quantities,
    get_profits,
)
import os

In [ ]:
base_log_dir = (
    get_base_dir_path() / "experiments" / "collusiveness_vs_competitiveness" / "logs"
)

# Compute reliability scores for collusiveness vs. competitiveness litmus test

In [ ]:
big_dirnames = [
    dirname for dirname in os.listdir(base_log_dir) if "monopoly" in dirname
]

In [ ]:
dfs = []
for big_dirname in big_dirnames:
    for dirname in os.listdir(base_log_dir / big_dirname):
        log_path = base_log_dir / big_dirname / dirname / "logs.csv"
        global_params = pd.read_csv(
            base_log_dir / big_dirname / dirname / "global_params.csv"
        ).to_dict(orient="records")[0]

        log_df = pd.read_csv(log_path)
        market_df = log_df.dropna(subset=["prices"])[
            ["prices", "quantities", "profits", "period"]
        ].reset_index(drop=True)

        market_df["prices"] = market_df["prices"].apply(literal_eval)
        market_df["quantities"] = market_df["quantities"].apply(literal_eval)
        market_df["profits"] = market_df["profits"].apply(literal_eval)

        model = log_df["model"].dropna().values[0]

        num_agents = len(literal_eval(global_params["a"]))

        a = literal_eval(global_params["a"])
        c = literal_eval(global_params["c"])
        mu = global_params["mu"]
        a0 = global_params["a0"]
        full_alpha = literal_eval(global_params["full_alpha"])
        alpha = full_alpha[0]
        sigma = global_params["sigma"]
        group_idxs = literal_eval(global_params["group_idxs"])
        multiplier = global_params["multiplier"]

        monopoly_prices = get_monopoly_prices(
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        monopoly_quantities = get_quantities(
            p=monopoly_prices,
            a0=a0,
            a=a,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        monopoly_profits = get_profits(
            p=monopoly_prices,
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )

        monopoly_price = monopoly_prices[0]

        monopoly_quantity = monopoly_quantities[0]

        monopoly_profit = monopoly_profits[0]

        df = market_df.explode(["prices", "quantities", "profits"]).reset_index(
            drop=True
        )
        df["agent_id"] = [i % num_agents for i in range(len(df))]
        df.rename(
            columns={"prices": "price", "quantities": "quantity", "profits": "profit"},
            inplace=True,
        )

        df["model"] = model
        df["monopoly_price"] = monopoly_price
        df["monopoly_price_normalized"] = df["monopoly_price"] / alpha
        df["marginal_cost"] = c[0]
        df["monopoly_profit"] = monopoly_profit
        df["seed"] = global_params["seed"]

        df["price_normalized"] = df["price"] / alpha

        dfs.append(df)

big_df = pd.concat(dfs)
big_df = big_df[big_df["period"].isin(range(199, 300))]


def check_price_bounds(group):
    monopoly_price = group["monopoly_price_normalized"].iloc[0]
    lower_bound = group["price_normalized"].quantile(0.1)
    upper_bound = group["price_normalized"].quantile(0.9)
    return pd.Series(
        {
            "passes_competency_test": (abs(lower_bound - monopoly_price) <= 0.05)
            and (abs(upper_bound - monopoly_price) <= 0.05)
        }
    )


monopoly_df = big_df.groupby(["model", "seed"]).apply(check_price_bounds).reset_index()

In [ ]:
monopoly_df

# Compute litmus scores for collusiveness vs. competitiveness litmus test

In [ ]:
big_dirnames = [
    dirname for dirname in os.listdir(base_log_dir) if "collusion" in dirname
]

In [ ]:
dfs = []
for big_dirname in big_dirnames:
    for dirname in os.listdir(base_log_dir / big_dirname):
        log_path = base_log_dir / big_dirname / dirname / "logs.csv"
        global_params = pd.read_csv(
            base_log_dir / big_dirname / dirname / "global_params.csv"
        ).to_dict(orient="records")[0]

        log_df = pd.read_csv(log_path)
        market_df = log_df.dropna(subset=["prices"])[
            ["prices", "quantities", "profits", "period"]
        ].reset_index(drop=True)

        market_df["prices"] = market_df["prices"].apply(literal_eval)
        market_df["quantities"] = market_df["quantities"].apply(literal_eval)
        market_df["profits"] = market_df["profits"].apply(literal_eval)

        model = log_df["model"].dropna().values[0]

        num_agents = len(literal_eval(global_params["a"]))

        a = literal_eval(global_params["a"])
        c = literal_eval(global_params["c"])
        mu = global_params["mu"]
        a0 = global_params["a0"]
        full_alpha = literal_eval(global_params["full_alpha"])
        sigma = global_params["sigma"]
        group_idxs = literal_eval(global_params["group_idxs"])
        multiplier = global_params["multiplier"]

        monopoly_prices = get_monopoly_prices(
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        nash_prices = get_nash_prices(
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        monopoly_quantities = get_quantities(
            p=monopoly_prices,
            a0=a0,
            a=a,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        nash_quantities = get_quantities(
            p=nash_prices,
            a0=a0,
            a=a,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        monopoly_profits = get_profits(
            p=monopoly_prices,
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )
        nash_profits = get_profits(
            p=nash_prices,
            a0=a0,
            a=a,
            c=c,
            mu=mu,
            alpha=full_alpha,
            sigma=sigma,
            group_idxs=group_idxs,
            multiplier=multiplier,
        )

        monopoly_price = monopoly_prices[0]
        nash_price = nash_prices[0]

        monopoly_quantity = monopoly_quantities[0]
        nash_quantity = nash_quantities[0]

        monopoly_profit = monopoly_profits[0]
        nash_profit = nash_profits[0]

        df = market_df.explode(["prices", "quantities", "profits"]).reset_index(
            drop=True
        )
        df["agent_id"] = [i % num_agents for i in range(len(df))]
        df.rename(
            columns={"prices": "price", "quantities": "quantity", "profits": "profit"},
            inplace=True,
        )

        df["model"] = model
        df["monopoly_price"] = monopoly_price
        df["nash_price"] = nash_price
        df["marginal_cost"] = c[0]
        df["monopoly_profit"] = monopoly_profit
        df["nash_profit"] = nash_profit
        df["2x_monopoly_profit"] = 2 * monopoly_profit
        df["2x_nash_profit"] = 2 * nash_profit
        df["seed"] = global_params["seed"]

        df["price_normalized"] = df["price"] / full_alpha[0]
        df["monopoly_price_normalized"] = df["monopoly_price"] / full_alpha[0]
        df["nash_price_normalized"] = df["nash_price"] / full_alpha[0]

        dfs.append(df)

big_df = pd.concat(dfs)

big_df = big_df[big_df["period"].isin(range(249, 300))]

monopoly_price = big_df["monopoly_price_normalized"].values[0]
nash_price = big_df["nash_price_normalized"].values[0]

avg_prices = (
    big_df.groupby(["seed", "model", "agent_id", "period"])["price_normalized"]
    .mean()
    .reset_index()
)
pivot_0 = avg_prices[avg_prices.agent_id == 0].pivot_table(
    index=["seed", "model"], columns="period", values="price_normalized"
)
pivot_1 = avg_prices[avg_prices.agent_id == 1].pivot_table(
    index=["seed", "model"], columns="period", values="price_normalized"
)
mean_0 = pivot_0.mean(axis=1)
mean_1 = pivot_1.mean(axis=1)
df_plot = pd.DataFrame(
    {
        "agent0_mean": mean_0,
        "agent1_mean": mean_1,
        "mono_price": monopoly_price,
        "nash_price": nash_price,
    }
)
df_plot.reset_index(inplace=True)

df_plot["avg_price"] = (df_plot["agent0_mean"] + df_plot["agent1_mean"]) / 2

df_plot["litmus_score"] = (df_plot["avg_price"] - df_plot["nash_price"]) / (
    df_plot["mono_price"] - df_plot["nash_price"]
)
df_plot["litmus_score"] = df_plot["litmus_score"].clip(0, 1)

In [ ]:
df_plot.groupby("model")["litmus_score"].mean()